In [ ]:
import cv2
import time
import PIL.Image
from io import BytesIO
import IPython.display

import pyrealsense2 as rs
import numpy as np

In [ ]:
#image_recorder = ImageRecorder(stream=video)
#video = cv2.VideoCapture(2)


# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

display2 = IPython.display.display('', display_id=2)

# Start streaming
pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
       
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
       
        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))
       
        f = BytesIO()
        PIL.Image.fromarray(images).save(f, 'jpeg')
        img = IPython.display.Image(data=f.getvalue())
        display2.update(img)
        time.sleep(0.20)
       
       
       
        # Show images
        #cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        #cv2.imshow('RealSense', images)
        #cv2.waitKey(1)
       
        #IPython.display.display(PIL.Image.fromarray(images))
        #IPython.display.clear_output()

finally:

    #video.release()
    # Stop streaming
    pipeline.stop()

In [ ]:
### TEST 1: Picture (might be dark, that is fine) ###
video = cv2.VideoCapture(2)
display1 = IPython.display.display('', display_id=1)
frame = video.read()[1]
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
f = BytesIO()
PIL.Image.fromarray(frame).save(f, 'jpeg')
img = IPython.display.Image(data=f.getvalue())
display1.update(img)
video.release()

In [ ]:
### TEST 2: Video (slow frame rate and ~2 second delay is fine) ###
video = cv2.VideoCapture(2)
video.set(3,320) # width
video.set(4,240) # height
display2 = IPython.display.display('', display_id=2)
start_time = time.time()
time_limit = 10
while time.time() - start_time < time_limit:
    frame = video.read()[1]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    f = BytesIO()
    PIL.Image.fromarray(frame).save(f, 'jpeg')
    img = IPython.display.Image(data=f.getvalue())
    display2.update(img)
    time.sleep(0.25)
video.release()